In [ ]:
from sage.all import *
import random

In [ ]:
def find_q(n):
    if not  n in [32,64,128,256,512,1024,2048]:
        raise ValueError("improper argument ",n)
        
    q = 1 + 2*n
    while True:
        if (q).is_prime():
            return q
        q += 2*n
    

In [ ]:
def ntt(f,N,xi,F):                            
    def _expand_(f): 
        u = f.list()
        return u + [0]*(N-len(u)) 
    
    def _ntt_(xi,N,f):
        if N==1:
            return f
        N_ = N/2 ; xi2 =  xi^2  
        f0 = [f[2*i]   for i in range(N_)] ; f1 = [f[2*i+1] for i in range(N_)] 
        ff0 = _ntt_(xi2,N_,f0) ; ff1 = _ntt_(xi2,N_,f1)  

        s  = xi ; ff = [F(0) for i in range(N)] 
        for i in range(N_):
            a = ff0[i] ; b = s*ff1[i]  
            ff[i] = a + b ; ff[i + N_] = a - b 
            s = s * xi2                     
        return ff 
    
    return _ntt_(xi,N,_expand_(f))

In [ ]:
def ntt_inv(ff,N,base):                             
    return sum([ff[i]*base[i] for i in range(N)])

In [ ]:
def random_pol(R,args=None):
    return R.random_element(args)

In [ ]:
N = int(input("Enter N: "))
q = find_q(N)
print("q = ",q)


In [ ]:
F = FiniteField(q)   
R = PolynomialRing(F, name="w")
w = R.gen()
g = (w^N + 1)
xi = g.roots(multiplicities=False)[-1]
rs = [xi^(2*i+1) for i in range(N)] 
base = crt_basis([(w - r) for r in rs]) 

In [ ]:
f = random_pol(R,N-random.randint(1,N-1))
print("f = ",end='')
for i in range(N-1):
    print(f"{f[i]}*w^{i} + ",end='')
print(f"{f[N-1]}*w^{N-1}")

In [ ]:
ff = ntt(f,N,xi,F)
print("ff = ",end='')
for i in range(N-1):
    print(f"{ff[i]}*w^{i} + ",end='')
print(f"{ff[N-1]}*w^{N-1}")

In [ ]:

fff = ntt_inv(ff,N,base)
print("fff = ",end='')
for i in range(N-1):
    print(f"{fff[i]}*w^{i} + ",end='')
print(f"{fff[N-1]}*w^{N-1}")

print("Correto ? ",f == fff) 